# Parameter identification interface
### User-interface to perform parameter identification of bioscrape/SBML models

* Simple functions to import models/data 
* Use the fast deterministic and stochastic simulators available within bioscrape
* (Optionally) Specify paramter priors, desired likelihood/cost functions 
* Create your own likelihood functions interface

## Loglikelihood Functions
Bioscrape comes with a number of built in log-likelihood functions for deterministic or stochastic parameter inference. These functions are detailed in the following notebook. First, a simple model is made to test the functions:

$\emptyset \xrightarrow[]{k_1} X \; \; \; \; X \xrightarrow[]{d_1} \emptyset$

A set of N trajectories is then generated from this model, from either the same or different initial conditions, across the same or different time-windows. Gaussian noise is then added to all the samples.

In [7]:
import bioscrape as bs
from bioscrape.types import Model
from bioscrape.simulator import py_simulate_model

import numpy as np
import pylab as plt

# Import bioscrape XML / bioscrape model object M / SBML file
# M  = bs.types.read_model_from_sbml(filename)
# M = Model('models/pid_model.xml')

# Or...Create a Model using the bioscrape API
species = ['X']
reactions = [(['X'], [], 'massaction', {'k':'d1'}), ([], ['X'], 'massaction', {'k':'k1'})]
k1 = 10.0
d1 = .2
params = [('k1', k1), ('d1', d1)]
initial_condition = {'X':0}
M = Model(species = species, reactions = reactions, parameters = params, 
          initial_condition_dict = initial_condition)


# Import data from CSV
# data = import_timeseries('test_data.csv', time_column = 2, value_column = 4, properties = {3 : 51})

M.py_initialize()

N = 10 #Number of trajectories
nT = 50 #number of timepoints
noise_std = 5 #Standar deviation of the guassian noise added onto the measurements

MultipleTimepoints = True #Different timepoints for each trajectory?
timepoint_list = []
timepoints = np.linspace(np.random.randint(0, 10), np.random.randint(10, 100), nT)

#Generate Trajectories
R = [] #Results as Pandas Dataframes
Data = [] #Results will become a numpy array
MultipleInitialConditions = True #Different initial conditions for each trajectory?

X0_list = [] #multiple initial conditions will be saved for inference
for n in range(N):
    if MultipleInitialConditions:
        initial_condition = {'X': np.random.randint(0, 100)}
        X0_list.append(initial_condition)
        
    if MultipleTimepoints:
        timepoints = np.linspace(np.random.randint(0, 10, 1), np.random.randint(10, 100, 1), num = 50).flatten()
        timepoint_list.append(timepoints)
    
    M.set_species(initial_condition)
    r = py_simulate_model(timepoints, Model = M, stochastic = True)
    R.append(r)
    noisy_data = r['X'].to_numpy() + np.random.normal(loc = 0, scale = noise_std, size = nT)
    Data.append(noisy_data)

#Plot Results
# plt.figure()
Data = np.array(Data)
# for n in range(N):
#     plt.plot(R[n]['time'], R[n]['X'], color = (n/N, 0, 1-n/N))
#     plt.plot(R[n]['time'], Data[n], 'o', color = (n/N, 0, 1-n/N))
#     plt.xlabel('time')
#     plt.ylabel('X')

In [20]:
M.get_params2index()

{'d1': 0, 'k1': 1}

In [4]:
from bioscrape.StochasticInference import *
spid = StochasticInference()
spid.M = M
spid.prior = {'k1' : [1e-3, 1e3],'d1' : [1e-2, 1e5]}
spid.params_to_estimate = {'k1':10, 'd1':0.2}
spid.prepare_mcmc(params = spid.params_to_estimate, prior = spid.prior, timepoints = timepoints,
                                         exp_data = Data, nwalkers = 8, nsteps = 50, nsamples = 10, 
                                         measurements = ['X'], initial_conditions = X0_list)
spid.run_mcmc()

[53.94187342 55.92248774 55.40916567 53.95098788 55.96558654 63.14110472
 61.64348635 52.24947122 55.52705012 60.70769666 64.87413151 41.46374371
 45.57084715 49.55427527 43.47890889 37.46190952 45.93045349 52.43860848
 52.88666115 45.99048165 52.15813748 44.1928127  56.35601302 42.51833994
 42.21818659 50.57185147 33.30939477 54.05586772 47.57913061 45.05291329
 39.13970131 50.06076574 49.83224492 53.9028091  52.08557722 54.81766559
 40.53167581 54.96342276 54.85129114 58.02253269 51.03961308 48.52440883
 52.8421914  60.64764283 65.77100982 66.85833578 68.74617475 58.82538409
 57.72484906 50.89656518]
['X']
50


ValueError: Second dimension of measurments must be the same length as measured_species

### To run the MCMC algorithm to identify parameters from the data, the following code can be used.

In [ ]:
from bioscrape.StochasticInference import *

spid = StochasticInference()
# Specify priors (uniform prior only for now)
spid.prior = {'k1' : [1e-3, 1e3],'d1' : [1e-2, 1e5]}


# Initial guesses for the parameters to estimate
spid.params_to_estimate = {'k1':10, 'd1':0.2}
spid.prepare_mcmc(params = spid.params_to_estimate, prior = spid.prior, timepoints = timepoints,
                                         exp_data = noisy_data, nwalkers = 8, nsteps = 50, nsamples = 10, 
                                         measurements = ['X'])
spid.run_mcmc(plot_show = True)
# fit_model is with identified parameters' "best" value substituted


In [ ]:
len(spid.measurements)

## Data Objects and Loglikelihood Objects for Trajectories
Different kinds of data must be stored in specific kinds of data objects. In turn, each Log-likelihood object is designed to accept a single kind of data object.

DeterministicLikelihood takes a BulkData Data Object. This likelihood sums up the L-norm between N experimental bulk trajectories and a single deterministic ODE simulation for each experimental trajectory.



StochasticTrajectoriesLikelihood takes a StochasticTrajectories Data object. This likelihood sums up the pair-wise L-norm between N experimental stochastic trajectories and N_simulations SSA (non-volume) simulations per experimental trajectory.

### StochasticTrajectories Constructor
*  Timepoints: when the data is sampled. Simulations will return results along the same timepoints.
 *   If each data sample has unique timepoints, timepoints may be passed in as a 2D array of size N x T.
 *    If all data is collected along the same range and number of timepoints, a single vector of T timepoints can be used
*  Data: an N x T x M array. N: number of samples. T: number of timepoints. M: number of species measured
* measured_species: list of M species names corresponding to model species and measurements.
* N: Number of samples.

### StochasticTrajectoriesLikelihood Constructor
* model: A Bioscrape Model
* data: StochasticTrajectories Data Object. If not passed in, can be added via .set_data(StochastiTrajectories) method.
#### Optional:
 * init_state: an initial state dictionary or a list of N (#samples) initial state dictionaries. Defaults to Model's initial state
 * N_simulations: Number of simulations per sample. Defaults to 1.
 * norm_order: What L-norm to use. Defaults to 1.
 * ModelCSimInterface. Generated from Model otherwise.
 * Simulator: How to simulate the model. Defaults to SSASimulator or DelaySSASimulator otherwise.
 
 

In [ ]:
#Create LogLikelihoodFunction
def get_likelihood_function(self)
    from bioscrape.inference import DeterministicLikelihood as DLL
    from bioscrape.inference import BulkData
    from bioscrape.inference import StochasticTrajectoriesLikelihood as STLL
    from bioscrape.inference import StochasticTrajectories


    # Create a Data Objects
    if MultipleTimepoints:
        DataStoch = StochasticTrajectories(np.array(timepoint_list), Data, ["X"], N)
        DataDet = BulkData(np.array(timepoint_list), Data, ["X"], N)
    else:
        DataStoch = StochasticTrajectories(timepoints, Data, ["X"], N)
        DataDet = BulkData(timepoints, Data, ["X"], N)

    #Ceate Likelihood objects:
    N_simulations = 3 #Number of simulations per sample to compare to
    norm_order = 1 #(integer) Which norm to use: 1-Norm, 2-norm, etc.

    #If there are multiple initial conditions in a data-set, should correspond to multiple initial conditions for inference.
    #Note len(X0_list) must be equal to the number of trajectories N
    if MultipleInitialConditions:    
        LL_stoch = STLL(model = M, init_state = X0_list, data = DataStoch, N_simulations = N_simulations, norm_order = norm_order)
        LL_det = DLL(model = M, init_state = X0_list, data = DataDet, norm_order = norm_order)


    #Multiple samples with a single initial only require a single initial condition.
    else:
        LL_stoch = STLL(model = M, init_state = initial_condition, data = DataStoch, norm_order = norm_order, N_simulations = N_simulations)
        LL_det = DLL(model = M, init_state = initial_condition, data = DataDet, norm_order = norm_order)

    print("LL_stoch.N_simulations", LL_stoch.py_get_N_simulations(), "LL_stoch.norm_order", LL_stoch.py_get_norm_order(), "LL_det.norm_order", LL_det.py_get_norm_order())


In [ ]:
npoints = 15
d_list = np.logspace(-12, 12, base = 2, num = npoints)
k_list = np.logspace(-12, 12, base = 2, num = npoints)
HM_stoch = np.zeros((len(d_list), len(k_list)))
HM_det = np.zeros((len(d_list), len(k_list)))


for di in range(len(d_list)):
    print("di=", di, end = "...ki =")
    for ki in range(len(k_list)):
        print(ki, end = " ")
        LL_stoch.set_init_params({"d1":d_list[di], "k1":k_list[ki]})
        LL_det.set_init_params({"d1":d_list[di], "k1":k_list[ki]})
        
        vs = LL_stoch.py_log_likelihood()
        vd = LL_det.py_log_likelihood()
        
        HM_stoch[di, ki] = -vs
        HM_det[di, ki] = -vd

plt.figure(figsize = (18, 8))
plt.subplot(121)
plt.title("Stochastic Log Likelihood of a Non-Identifiable Manifold\n k ="+str(k1)+" d="+str(d1))
plt.xlabel("log k")
plt.xticks(range(npoints), [round(np.log(k), 1) for k in k_list])
plt.ylabel("log d")
plt.yticks(range(npoints), [round(np.log(d), 1) for d in d_list])
cb = plt.pcolor(np.log(HM_stoch))
_ = plt.colorbar(cb)

plt.subplot(122)
plt.title("Deterministic Log Likelihood of a Non-Identifiable Manifold\n k ="+str(k1)+" d="+str(d1))
plt.xlabel("log k")
plt.xticks(range(npoints), [round(np.log(k), 1) for k in k_list])
plt.ylabel("log d")
plt.yticks(range(npoints), [round(np.log(d), 1) for d in d_list])
cb = plt.pcolor(np.log(HM_det))
_ = plt.colorbar(cb)